# Document Retrival with Title Embedding and IDF on Texts (DR.TEIT)

In this method we used two scoring measure and aggregate them by a convex combination as below:
$$
λ*Similiarity_{Title Embedding} + (1-λ)*Similiarity_{TextIDF}
$$

We used LaBSE model for out embeddings. For computing title embedding similarities we used cosine similarity between query embeddings and each document's title embedding.

For the second part we used character-level (2gram to 8gram). We also trained our TF-IDF transformation matrix on the Multidoc2dial2022 documnets.

## Dataset
### Dataset Description

- **mutldoc2dial_doc.json** contains the documents that are indexed by key `domain` and `doc_id` . Each document instance includes the following,

  - `doc_id`: the ID of a document;
  - `title`: the title of the document;
  - `domain`: the domain of the document;
  - `doc_text`: the text content of the document (without HTML markups);
  - `doc_html_ts`: the document content with HTML markups and the annotated spans that are indicated by `text_id` attribute, which corresponds to `id_sp`.
  - `doc_html_raw`: the document content with HTML markups and without span annotations.
  - `spans`: key-value pairs of all spans in the document, with `id_sp` as key. Each span includes the following,
    - `id_sp`: the id of a  span as noted by `text_id` in  `doc_html_ts`;
    - `start_sp`/  `end_sp`: the start/end position of the text span in `doc_text`;
    - `text_sp`: the text content of the span.
    - `id_sec`: the id of the (sub)section (e.g. `<p>`) or title (`<h2>`) that contains the span.
    - `start_sec` / `end_sec`: the start/end position of the (sub)section in `doc_text`.
    - `text_sec`: the text of the (sub)section.
    - `title`: the title of the (sub)section.
    - `parent_titles`: the parent titles of the `title`.

- **multidoc2dial_dial_train.json** and **multidoc2dial_dial_validation.json**  contain the training and dev split of dialogue data that are indexed by key `domain` . Please note: **For test split, we only include a dummy file in this version.**

  Each dialogue instance includes the following,

  - `dial_id`: the ID of a dialogue;
  - `turns`: a list of dialogue turns. Each turn includes,
    - `turn_id`: the time order of the turn;
    - `role`: either "agent" or "user";READ
    - `da`: dialogue act;
    - `references`: a list of spans with `id_sp` ,  `label` and `doc_id`. `references` is empty if a turn is for indicating previous user query not answerable or irrelevant to the document. **Note** that labels "*precondition*"/"*solution*" are fuzzy annotations that indicate whether a span is for describing a conditional context or a solution.
    - `utterance`: the human-generated utterance based on the dialogue scene.
Downloading the training dataset:

In [1]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [2]:
!gdown --id 1Ln4pU93_ofAkbrz1uibsNABB0QsEaOXw

/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Ln4pU93_ofAkbrz1uibsNABB0QsEaOXw
To: /content/multidoc2dial.zip
100% 6.45M/6.45M [00:00<00:00, 34.6MB/s]


In [3]:
!unzip multidoc2dial.zip

Archive:  multidoc2dial.zip
   creating: multidoc2dial/
  inflating: multidoc2dial/multidoc2dial_dial_validation.json  
  inflating: multidoc2dial/multidoc2dial_dial_train.json  
  inflating: multidoc2dial/multidoc2dial_dial_test.json  
  inflating: multidoc2dial/multidoc2dial_doc.json  
  inflating: multidoc2dial/README.md  


In [4]:
def clean_text(text):
    """
    Clean the given text.

    :param text: input text
    :type text: str
    :return: cleaned string
    """
    return text.strip()

In [5]:
import json
with open('multidoc2dial/multidoc2dial_doc.json', 'r') as f:
    multidoc2dial_doc = json.load(f)

### Extracting titles

In [6]:
parent_titles = []
titles = []
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        
        for id_sp1 in multidoc2dial_doc['doc_data'][doc_idx1][doc_idx2]['spans']:
            titles.append(multidoc2dial_doc['doc_data'][doc_idx1][doc_idx2]['spans'][id_sp1]['title']) 
            parent_titles.append(doc_idx2)
          
titles


['Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefits Planner: Survivors | Planning For Your Survivors',
 'Benefi

In [7]:
len(titles)

35659

### Extracting document texts

In [8]:
doc_texts_train = []
title_to_domain = {}
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        title_to_domain[doc_idx2] = doc_idx1
        for id_sp1 in multidoc2dial_doc['doc_data'][doc_idx1][doc_idx2]['spans']:
             s = (multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['spans'][id_sp1]['text_sec'].strip())
             s = s.replace('!', '.')
             s = s.replace('! ', '.')
             s = s.replace('?', '.')
             s = s.replace('? ', '.')
             s = s.replace('\n', '.')
             m = s.split('.')
             #print(m)
             S = ""
             func = lambda w: w[:1].lower() + w[1:] if w else ''
             for word in m:
                 S = S + func(word) + ' '
             doc_texts_train.append(S)
            
doc_texts_train[10]


'the number of credits needed to provide benefits for your survivors depends on your age when you die  No one needs more than 40 credits 10 years of work to be eligible for any Social Security benefit  But , the younger a person is , the fewer credits they must have for family members to receive survivors benefits  '

In [9]:
len(doc_texts_train)

35659

## Encoding the sentences
We use the LaBSE which is a Language-agnostic BERT Sentence Encoder (LaBSE) is a BERT-based model trained for sentence embedding for 109 languages. The pre-training process combines masked language modeling with translation language modeling. The model is useful for getting multilingual sentence embeddings and for bi-text retrieval.

In [10]:
!pip install --quiet transformers

     |████████████████████████████████| 5.8 MB 34.1 MB/s 
     |████████████████████████████████| 7.6 MB 68.3 MB/s 
     |████████████████████████████████| 182 kB 87.6 MB/s 


In [11]:
from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForSequenceClassification
import numpy as np
import torch
from torch.nn.functional import normalize

from tqdm import tqdm

model_name = "setu4993/LaBSE"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.22M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.62M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

In [12]:
tokenizer_labse = AutoTokenizer.from_pretrained("setu4993/LaBSE")
model_labse = AutoModel.from_pretrained("setu4993/LaBSE")

### `get_embeddings`
In this method we extract the **pooler output** (Last layer hidden-state of the first token of the sequence (classification token) after further processing through the layers used for the auxiliary pretraining task. E.g. for BERT-family of models, this returns the classification token after processing through a linear layer and a tanh activation function. The linear layer weights are trained from the next sentence prediction (classification) objective during pretraining).

In [13]:
def get_embeddings(sentece):
    """
    Return embeddings based on encoder model

    :param sentence: input sentence(s)
    :type sentence: str or list of strs
    :return: embeddings
    """
    tokenized = tokenizer_labse(sentece,
                                return_tensors="pt",
                                padding=True)
    with torch.no_grad():
        embeddings = model_labse(**tokenized)
    
    return np.squeeze(np.array(embeddings.pooler_output))

### Title embedding

In [14]:
title_embeddings = []
progress = 0
TRAIN_SIZE = len(titles)
for title in titles:
    title_embeddings.append(get_embeddings(title))
    progress += 1
    if progress % 50 == 0:
        print('Progress Percent = {}%'.format(100 * progress / TRAIN_SIZE))

Progress Percent = 0.14021705600269216%
Progress Percent = 0.2804341120053843%
Progress Percent = 0.4206511680080765%
Progress Percent = 0.5608682240107686%
Progress Percent = 0.7010852800134608%
Progress Percent = 0.841302336016153%
Progress Percent = 0.9815193920188452%
Progress Percent = 1.1217364480215373%
Progress Percent = 1.2619535040242296%
Progress Percent = 1.4021705600269216%
Progress Percent = 1.542387616029614%
Progress Percent = 1.682604672032306%
Progress Percent = 1.822821728034998%
Progress Percent = 1.9630387840376904%
Progress Percent = 2.1032558400403825%
Progress Percent = 2.2434728960430745%
Progress Percent = 2.383689952045767%
Progress Percent = 2.523907008048459%
Progress Percent = 2.664124064051151%
Progress Percent = 2.8043411200538433%
Progress Percent = 2.9445581760565354%
Progress Percent = 3.084775232059228%
Progress Percent = 3.22499228806192%
Progress Percent = 3.365209344064612%
Progress Percent = 3.505426400067304%
Progress Percent = 3.645643456069996

In [16]:
with open('doc_title_LaBSE_Embedding.npy', 'wb') as f:
    np.save(f, np.array(title_embeddings))

In [ ]:
title_to_embeddings = {}
progress = 0
TRAIN_SIZE = len(titles)
for title in titles:
    title_to_embeddings[title] = get_embeddings(title)
    progress += 1
    if progress % 50 == 0:
        print('Progress Percent = {}%'.format(100 * progress / TRAIN_SIZE))

Progress Percent = 0.14021705600269216%
Progress Percent = 0.2804341120053843%
Progress Percent = 0.4206511680080765%
Progress Percent = 0.5608682240107686%
Progress Percent = 0.7010852800134608%
Progress Percent = 0.841302336016153%
Progress Percent = 0.9815193920188452%
Progress Percent = 1.1217364480215373%
Progress Percent = 1.2619535040242296%
Progress Percent = 1.4021705600269216%
Progress Percent = 1.542387616029614%
Progress Percent = 1.682604672032306%
Progress Percent = 1.822821728034998%
Progress Percent = 1.9630387840376904%
Progress Percent = 2.1032558400403825%
Progress Percent = 2.2434728960430745%
Progress Percent = 2.383689952045767%
Progress Percent = 2.523907008048459%
Progress Percent = 2.664124064051151%
Progress Percent = 2.8043411200538433%
Progress Percent = 2.9445581760565354%
Progress Percent = 3.084775232059228%
Progress Percent = 3.22499228806192%
Progress Percent = 3.365209344064612%
Progress Percent = 3.505426400067304%
Progress Percent = 3.645643456069996

In [17]:
import pickle
with open('title_to_embeddings.pkl', 'wb') as f:
    pickle.dump(title_to_embeddings, f)

## Calculating the IDF for each token

In [18]:
words = set()
doc_texts_train_tokenized = []
for doc in doc_texts_train:
    tokenized_doc = [s.lower() for s in tokenizer_labse.tokenize(doc)]
    doc_texts_train_tokenized.append(tokenized_doc) 
    words = set(tokenized_doc).union(words)
len(words)

8477

In [19]:
words2IDF = {}
N_doc = len(doc_texts_train)
for i, word in enumerate(words):
    n_word = 0
    for doc in doc_texts_train_tokenized:
        if word in doc:
            n_word += 1
    words2IDF[word] = np.log(N_doc / (n_word + 1))
    if i % 1000 == 0:
        print(word, words2IDF[word])

##di 7.3462626324699905
disable 4.944422581380603
news 7.437234410675717
top 4.94836735967162
neurological 7.537317869232699
planting 7.648543504342924
noce 8.28453227106292
modified 6.720556732705578
spam 6.9852492869326595


In [20]:
len(words2IDF)

8477

In [113]:
def calc_idf_score(sentence):
    """
    Calculate the mean idf score for given sentence.

    :param sentence: input sentence
    :type sentence: str
    :return: mean idf score of sentence token
    """
    tokenzied_sentence = [s.lower() for s in tokenizer_labse.tokenize(sentence)]
    score = 0
    for token in tokenzied_sentence:
        if token in words2IDF:
            score += words2IDF[token]
        else:
            score += np.log(N_doc)
    if (len(tokenzied_sentence)==0):
        return 0
    else:
        return score / len(tokenzied_sentence)

### Saving the IDF values dictionary

In [22]:
import pickle
with open('IDFs.pkl', 'wb') as f:
    pickle.dump(words2IDF, f)

**make a dictionary out of docs**


In [90]:
Docs = {}
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        
        for id_sp1 in multidoc2dial_doc['doc_data'][doc_idx1][doc_idx2]['spans']:
            KEY2 = multidoc2dial_doc['doc_data'][doc_idx1][doc_idx2]['spans'][id_sp1]['id_sp']
            Docs[doc_idx2]={}
            Docs[doc_idx2][KEY2] = multidoc2dial_doc['doc_data'][doc_idx1][doc_idx2]['spans'][id_sp1]['title']
            print(doc_idx2,'  ',id_sp1, ' ', Docs[doc_idx2][id_sp1])
           

Streaming output truncated to the last 5000 lines.
Appeal a Credit Decision Demo | Extenuating Circumstances | Federal Student Aid#1_0    150   Proof of Extenuating Circumstances
Appeal a Credit Decision Demo | Extenuating Circumstances | Federal Student Aid#1_0    151   Proof of Extenuating Circumstances
Appeal a Credit Decision Demo | Extenuating Circumstances | Federal Student Aid#1_0    152   Proof of Extenuating Circumstances
Appeal a Credit Decision Demo | Extenuating Circumstances | Federal Student Aid#1_0    153   Proof of Extenuating Circumstances
Appeal a Credit Decision Demo | Extenuating Circumstances | Federal Student Aid#1_0    154   Proof of Extenuating Circumstances
Appeal a Credit Decision Demo | Extenuating Circumstances | Federal Student Aid#1_0    155   Proof of Extenuating Circumstances
Appeal a Credit Decision Demo | Extenuating Circumstances | Federal Student Aid#1_0    156   Proof of Extenuating Circumstances
Appeal a Credit Decision Demo | Extenuating Circumsta

# Constructing the Follow-up Dataset



In [95]:
def combine_sentences(s1, s2):
    separation_token = "  "
    return s1 + separation_token + s2


def construct_followup_dataset(filepath):
    import json
    with open(filepath, 'r') as f:
        multidoc2dial_dial_train = json.load(f)
    
    historys = []
    questions = []
    combined = []
    labels = []
    prev_docs = []
    current_docs = []
    prev_answers = []

    for domain in multidoc2dial_dial_train['dial_data']:
        for dial in multidoc2dial_dial_train['dial_data'][domain]:
            prev_doc = ''
            prev_question = ''
            prev_answer = ''
            for turn in dial['turns']:
                if turn['role'] == "user":
                    current_question = turn['utterance']
                    historys.append(prev_question)
                    questions.append(current_question)
                    
                    combined.append(combine_sentences(prev_question, current_question))

                    doc = turn['references'][0]['doc_id']
                    current_id_sep = turn['references'][0]['id_sp']
                    if(current_id_sep in Docs[doc]):
                        current_doc = Docs[doc][current_id_sep]
                    else:
                        current_doc = doc
                    labels.append(int(current_doc==prev_doc))

                    prev_docs.append(prev_doc)
                    current_docs.append(current_doc)
                    prev_answers.append(prev_answer)

                    prev_doc, prev_question = current_doc, current_question
                else:
                    prev_answer = turn['utterance']
                    
    return historys, questions, combined, labels, prev_docs, current_docs, prev_answers


In [15]:
import pandas as pd

train_history, train_questions, train_combined, train_labels, train_prev_docs, train_current_docs, train_prev_answers = construct_followup_dataset('/content/multidoc2dial/multidoc2dial_dial_train.json')
test_history, test_questions, test_combined, test_labels, test_prev_docs, test_current_docs, test_prev_answers = construct_followup_dataset('/content/multidoc2dial/multidoc2dial_dial_validation.json')

train_dict_dataset = {"history":train_history, "question": train_questions, "combined": train_combined, "followup": train_labels, "prev_doc": train_prev_docs, "current_doc": train_current_docs, "prev_answer": train_prev_answers}
test_dict_dataset = {"history":test_history, "question": test_questions, "combined": test_combined, "followup": test_labels, "prev_doc": test_prev_docs, "current_doc": test_current_docs, "prev_answer": test_prev_answers}

train_df = pd.DataFrame(train_dict_dataset)
test_df = pd.DataFrame(test_dict_dataset)

NameError: ignored

In [26]:
max([len(x["combined"].split()) for _, x in test_df.iterrows()])

73

In [27]:
train_df

,history,question,combined,followup,prev_doc,current_doc,prev_answer
0,,"Hello, I forgot o update my address, can you h...","Hello, I forgot o update my address, can you...",0,,Top 5 DMV Mistakes and How to Avoid Them#3_0,
1,"Hello, I forgot o update my address, can you h...",Can I do my DMV transactions online?,"Hello, I forgot o update my address, can you h...",1,Top 5 DMV Mistakes and How to Avoid Them#3_0,Top 5 DMV Mistakes and How to Avoid Them#3_0,"hi, you have to report any change of address t..."
2,Can I do my DMV transactions online?,You've got it. Another query about DMV. What h...,Can I do my DMV transactions online? You've g...,0,Top 5 DMV Mistakes and How to Avoid Them#3_0,Registration suspensions for failure to pay to...,"Yes, you can sign up for MyDMV for all the onl..."
3,You've got it. Another query about DMV. What h...,"Besides that, will I receive a notice?",You've got it. Another query about DMV. What h...,1,Registration suspensions for failure to pay to...,Registration suspensions for failure to pay to...,the suspension is placed on hold pending the o...
4,"Besides that, will I receive a notice?",If you submit the affidavit?,"Besides that, will I receive a notice? If you...",1,Registration suspensions for failure to pay to...,Registration suspensions for failure to pay to...,"the NYS Department of Motor Vehicles , "" DMV ..."
...,...,...,...,...,...,...,...
23394,"By the way, who can I contact to give me infor...",What if I've fallen behind on one or more loan...,"By the way, who can I contact to give me infor...",0,Loan Servicers | Federal Student Aid#1_0,Student Loan Repayment | Federal Student Aid#1_0,Your school's financial aid office must have t...
23395,What if I've fallen behind on one or more loan...,I have another question regarding the Military...,What if I've fallen behind on one or more loan...,0,Student Loan Repayment | Federal Student Aid#1_0,Student Loan Deferment | Federal Student Aid#1_0,One thing you definitely want to avoid is goin...
23396,I have another question regarding the Military...,something else I want to ask about FAFSA. What...,I have another question regarding the Military...,0,Student Loan Deferment | Federal Student Aid#1_0,Student Loan Repayment | Federal Student Aid#1_0,You will have to complete the Military Service...
23397,something else I want to ask about FAFSA. What...,How can I make a payment by post?,something else I want to ask about FAFSA. What...,1,Student Loan Repayment | Federal Student Aid#1_0,Student Loan Repayment | Federal Student Aid#1_0,contact your loan servicer to find out your op...


In [28]:
def tokenize_function(examples, prediction=False, cuda=False):
    if prediction:
        tokenized = tokenizer(examples['combined'], max_length=128, padding="max_length", truncation=True, return_tensors='pt')
    else:
        tokenized = tokenizer(examples['combined'], max_length=128, padding="max_length", truncation=True)
    if cuda:
        tokenized_cuda = {}
        for key, value in tokenized.items():
            tokenized_cuda[key] = value.cuda()
        return tokenized_cuda
    else:
        return tokenized

**dataloader**

In [30]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 4.7 MB/s 
     |████████████████████████████████| 132 kB 87.4 MB/s 
     |████████████████████████████████| 212 kB 97.3 MB/s 
     |████████████████████████████████| 127 kB 90.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [33]:
from datasets import Dataset, DatasetDict

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

tokenized_trainset = train_dataset.map(tokenize_function, batched=True)
tokenized_testset = test_dataset.map(tokenize_function, batched=True)

tokenized_trainset = tokenized_trainset.rename_column("followup", "label")
tokenized_testset = tokenized_testset.rename_column("followup", "label")

fud_dataset = DatasetDict()

fud_dataset['train'] = tokenized_trainset
fud_dataset['validation'] = tokenized_testset

  0%|          | 0/24 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

## Methods

### IDF only - Vanilla

This is the first method which we used for document retriver. In this method we just used similarity between query embeddign and document title embedding. We also used IDF scores as a factor for history queries by which we can pay more attention to the more informative query.

In [ ]:
def predict_labelwise_doc_at_history(queries, k=1):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """
    similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    for query in queries:
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        query_sim = np.array(query_sim)

        coef = calc_idf_score(query)
        coef_sum += coef
        similarities += coef * query_sim

    similarities = similarities / coef_sum
    best_k_idx = similarities.argsort()[::-1][:k]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    accuracy = similarities[best_k_idx]
    return accuracy, predictions

### IDF ordered

In the last method there wasn't any difference between query and histories - all sentences would be treat same regardless of their queried time. From now we will use a reweighting (multiplying query's score to $2^{-i}$ when $i$ is the index of query) system which favour last query more.

In [ ]:
def predict_labelwise_doc_at_history_ordered(queries, k=1):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """
    similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    for i, query in enumerate(queries):
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        query_sim = np.array(query_sim)

        coef = 2**(-i) * calc_idf_score(query)
        coef_sum += coef
        similarities += coef * query_sim

    similarities = similarities / coef_sum
    best_k_idx = similarities.argsort()[::-1][:k]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    accuracy = similarities[best_k_idx]
    return accuracy, predictions

### IDF ordered - softmaxed
In this method we changed a little bit. Instead of using coefitionts barely we apply the softmax function favouring the maximum score more. But results were not good. 

In [ ]:
def predict_labelwise_doc_at_history_ordered_softmaxed(queries, k=1):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """
    similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coefs = []
    sims = []
    for i, query in enumerate(queries):
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        sims.append(np.array(query_sim))
        coefs.append(2**(-i) * calc_idf_score(query))
    
    # Softmax:
    coefs = np.array(list(map(lambda x: np.exp(-x), coefs)))
    coefs /= coefs.sum()
    coefs = list(coefs)

    for coef, sim in zip(coefs, sims):
        similarities += coef * sim
    best_k_idx = similarities.argsort()[::-1][:k]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    accuracy = similarities[best_k_idx]
    return accuracy, predictions

### IDF + self attention (cosine sim)

In this method we changed the reweighting method from power-order ($2^{-i}$) to a feed-forward self-attention mechanism. Here we just simply use cosine similarity between each history and query as its coefficient. As it's showing how far is it from the main query.

In [ ]:
def predict_labelwise_doc_at_history_selfatt(queries, k=1):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """
    similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    query0_embd = get_embeddings(queries[0])
    for query in queries:
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        query_sim = np.array(query_sim)

        coef = calc_idf_score(query) * np.dot(query0_embd, query_embd) / (np.linalg.norm(query_embd) * np.linalg.norm(query0_embd))
        coef_sum += coef
        similarities += coef * query_sim

    similarities = similarities / coef_sum
    best_k_idx = similarities.argsort()[::-1][:k]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    accuracy = similarities[best_k_idx]
    return accuracy, predictions

### DR. TEIT*

In this method we used two scoring measure and aggregate them by a convex combination as below:
$$
λ*Similiarity_{Title Embedding} + (1-λ)*Similiarity_{TextIDF}
$$

We used LaBSE model for out embeddings. For computing title embedding similarities we used cosine similarity between query embeddings and each document's title embedding.

For the second part we used character-level (2gram to 8gram). We also trained our TF-IDF transformation matrix on the Multidoc2dial2022 documnets.

**NOTE: In `predict_DR_TEIT` you may see a diffrent notation (`alpha`) but they are the same.**

#### TF-IDF Transformation Matrix Fitting

In [34]:
doc_texts_train = []
title_to_domain = {}
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        title_to_domain[doc_idx2] = doc_idx1
        for id_sp1 in multidoc2dial_doc['doc_data'][doc_idx1][doc_idx2]['spans']:
             s = (multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['spans'][id_sp1]['text_sec'].strip())
             s = s.replace('!', '.')
             s = s.replace('! ', '.')
             s = s.replace('?', '.')
             s = s.replace('? ', '.')
             s = s.replace('\n', '.')
             m = s.split('.')
             #print(m)
             S = ""
             func = lambda w: w[:1].lower() + w[1:] if w else ''
             for word in m:
                 S = S + func(word) + ' '
             doc_texts_train.append(S)
            
doc_texts_train[10]

'the number of credits needed to provide benefits for your survivors depends on your age when you die  No one needs more than 40 credits 10 years of work to be eligible for any Social Security benefit  But , the younger a person is , the fewer credits they must have for family members to receive survivors benefits  '

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(strip_accents=None,
                                 analyzer='char',
                                 ngram_range=(4, 10),
                                 norm='l2',
                                 use_idf=True,
                                 smooth_idf=True)
tfidf_wm = tfidfVectorizer.fit_transform(doc_texts_train)

In [36]:
import pickle
with open('tfidfVectorizer.pkl', 'wb') as f:
    pickle.dump(tfidfVectorizer, f)

with open('tfidf_wm.pkl', 'wb') as f:
    pickle.dump(tfidf_wm, f)

#### DR. TEIT

In [116]:
def predict_DR_TEIT(queries, k=1, alpha=10):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """

    idf_score = np.array(list(map(lambda x: 0.0, title_embeddings)))
    tfidf_score = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    for i, query in enumerate(queries):
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        query_sim = np.array(query_sim)
        # coef = 2**(-i) * calc_idf_score(query)
        #print(query)
        coef = calc_idf_score(query)
        coef_sum += coef
     
        idf_score += coef * query_sim
        tfidf_score += coef * np.squeeze(np.asarray(tfidf_wm @ tfidfVectorizer.transform([query]).todense().T))

    scores = (idf_score + alpha * tfidf_score) / coef_sum
    best_k_idx = scores.argsort()[::-1][:k]
    scores = scores[best_k_idx]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    return (scores, predictions)

 **fudnet model**

In [41]:
model_name = "setu4993/LaBSE"

tokenizer = AutoTokenizer.from_pretrained(model_name)
fudnet_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

device = torch.device("cuda:0")
fudnet_model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at setu4993/LaBSE and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(501153, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [42]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-42-75db7450f822>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [43]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='/home/',
    num_train_epochs=2,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=50,
    weight_decay=0.01,
    logging_steps=10,
    evaluation_strategy='epoch',
    save_strategy ='epoch',
    load_best_model_at_end=True,
    # auto_find_batch_size=True,
)

trainer = Trainer(
    model=fudnet_model,
    args=training_args,
    train_dataset=fud_dataset['train'],
    eval_dataset=fud_dataset['validation'],
    compute_metrics=compute_metrics
)

In [97]:
trainer.train()

***** Running training *****
  Num examples = 23399
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1464
  Number of trainable parameters = 470928386
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: current_doc, combined, prev_answer, question, history, prev_doc. If current_doc, combined, prev_answer, question, history, prev_doc are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.159400,0.185650,0.935498
2,0.108800,0.195609,0.936610


***** Running Evaluation *****
  Num examples = 4496
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: current_doc, combined, prev_answer, question, history, prev_doc. If current_doc, combined, prev_answer, question, history, prev_doc are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
Saving model checkpoint to /home/checkpoint-732
Configuration saved in /home/checkpoint-732/config.json
Model weights saved in /home/checkpoint-732/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4496
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: current_doc, combined, prev_answer, question, history, prev_doc. If current_doc, combined, prev_answer, question, history, prev_doc are not expected by `BertForSequenceCl

TrainOutput(global_step=1464, training_loss=0.12270655028033452, metrics={'train_runtime': 358.0189, 'train_samples_per_second': 130.714, 'train_steps_per_second': 4.089, 'total_flos': 3093002011284480.0, 'train_loss': 0.12270655028033452, 'epoch': 2.0})

#FUDnet + Dr.TEIT

In [117]:
def predict_FUDNet_DR_TEIT(data, k=1):
    inputs = tokenize_function(data, prediction=True, cuda=True)
    outputs = fudnet_model(**inputs)
    is_followup = bool(torch.argmax(outputs.logits))
    
    if is_followup:
        dr_scores, dr_predictions = predict_DR_TEIT([data['prev_answer'], data['question'], data['history']], k=k)
        return dr_predictions
    else:
        dr_scores, dr_predictions = predict_DR_TEIT([data['question']], k=k)
        return dr_predictions

In [99]:
test_df.loc[2]

history                         Don't do that I'll get insurance
question       I have, that is why I am here to clear that up...
combined       Don't do that I'll get insurance  I have, that...
followup                                                       1
prev_doc            Top 5 DMV Mistakes and How to Avoid Them#3_0
current_doc         Top 5 DMV Mistakes and How to Avoid Them#3_0
prev_answer    Okay, have you received a letter from the DMV ...
Name: 2, dtype: object

In [103]:
test_df.loc[2].current_doc

'Top 5 DMV Mistakes and How to Avoid Them#3_0'

In [100]:
predict_FUDNet_DR_TEIT(test_df.loc[2], k=5)

['1. Forgetting to Update Address',
 'I registered my vehicle out of state, and then I received a letter from the DMV about a lapse of insurance on the vehicle. What can I do?',
 'I received a Letter that states my license is suspended. What can I do?',
 'I received a Letter that states my insurance lapsed. What can I do?',
 'Do I need insurance to register my vehicle?']

## Test
In the test dataset we just picked ones with **user** turn.

In [ ]:
test_queries = ["I'm looking for information regarding benefits planning, can you help me?",
                "I want to know about the benefits plan for survivors, can you give me more information about this?",
                "What are Social Security credits?"]
test_labels = ["Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0",
               "Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0",
               "Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0"]

In [ ]:
import json
with open('multidoc2dial/multidoc2dial_dial_train.json', 'r') as f:
    multidoc2dial_dial_train = json.load(f)

In [ ]:
doc_sentence_test = []
doc_label_test = []
for doc_idx1 in multidoc2dial_dial_train['dial_data']:
    for dial in multidoc2dial_dial_train['dial_data'][doc_idx1]:
        for turns in dial['turns']:
            if turns['role'] == "user":
                doc_sentence_test.append(turns['utterance'])
                doc_label_test.append(turns['references'][0]['doc_id'])

In [ ]:
TEST_SIZE = len(doc_sentence_test)
TEST_SIZE

23399

In [ ]:
TEST_SIZE = TEST_SIZE // 20   #   For making it faster

### IDF only - Vanilla

In [ ]:
accs, preds = predict_labelwise_doc_at_history([test_queries[2],
                                               test_queries[1],
                                               test_queries[0]],
                                               k=5)
print(accs)
print(preds)
print('-' * 20)

[0.37842299 0.37675699 0.37660415 0.37555711 0.37375754]
['Benefits Planner: Survivors | How You Apply | Social Security Administration#1_0', 'Benefits Planner: Survivors | How You Apply | Social Security Administration#2_0', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2', 'How To Apply For Social Security Disability Benefits#1_0']
--------------------


In [ ]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in range(2, TEST_SIZE):
    act_doc = doc_label_test[i]
    accs, preds = predict_labelwise_doc_at_history([doc_sentence_test[i],
                                                   doc_sentence_test[i-1],
                                                   doc_sentence_test[i-2]],
                                                   k=500)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

MRR: mean=0.11905440352084566, var=0.05986443862836907
Prec@(1) = 0.06 | Prec@(5) = 0.15 | Prec@(10) = 0.23 | Prec@(50) = 0.47 | Prec@(100) = 0.69 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100
MRR: mean=0.11182382229775921, var=0.04994596429538301
Prec@(1) = 0.045 | Prec@(5) = 0.155 | Prec@(10) = 0.25 | Prec@(50) = 0.5 | Prec@(100) = 0.715 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200
MRR: mean=0.1327195627539497, var=0.06172267295242753
Prec@(1) = 0.06 | Prec@(5) = 0.19 | Prec@(10) = 0.29 | Prec@(50) = 0.5366666666666666 | Prec@(100) = 0.76 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 300
MRR: mean=0.1627914543093306, var=0.07950550918949202
Prec@(1) = 0.085 | Prec@(5) = 0.2325 | Prec@(10) = 0.335 | Prec@(50) = 0.59 | Prec@(100) = 0.795 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 400
MRR: mean=0.18872528867358432, var=0.09459400049203383
Prec@(1) = 0.108 | Prec@(5) = 0.266 | Prec@(10) = 0.364 | Prec@(50) = 0.622 | Prec@(100) = 0.816 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 500
MRR: mean=0.1826

### IDF ordered

In [ ]:
accs, preds = predict_labelwise_doc_at_history_ordered([test_queries[2],
                                                        test_queries[1],
                                                        test_queries[0]],
                                                        k=5)
print(accs)
print(preds)
print('-' * 20)

[0.41813262 0.41414311 0.413184   0.41088731 0.40784173]
['Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#7_0_1_2', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3_4', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#12_0_1_2']
--------------------


In [ ]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in range(2, TEST_SIZE):
    act_doc = doc_label_test[i]
    accs, preds = predict_labelwise_doc_at_history_ordered([doc_sentence_test[i],
                                                            doc_sentence_test[i-1],
                                                            doc_sentence_test[i-2]],
                                                            k=500)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

MRR: mean=0.144511049798484, var=0.08118642286253466
Prec@(1) = 0.09 | Prec@(5) = 0.16 | Prec@(10) = 0.28 | Prec@(50) = 0.53 | Prec@(100) = 0.72 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100
MRR: mean=0.15031672380406347, var=0.08741299578833683
Prec@(1) = 0.1 | Prec@(5) = 0.165 | Prec@(10) = 0.275 | Prec@(50) = 0.545 | Prec@(100) = 0.76 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200
MRR: mean=0.17559390346365905, var=0.09533169069906142
Prec@(1) = 0.11 | Prec@(5) = 0.23 | Prec@(10) = 0.3333333333333333 | Prec@(50) = 0.5633333333333334 | Prec@(100) = 0.7833333333333333 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 300
MRR: mean=0.1914336875137787, var=0.09923098973353595
Prec@(1) = 0.115 | Prec@(5) = 0.2625 | Prec@(10) = 0.3625 | Prec@(50) = 0.605 | Prec@(100) = 0.8075 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 400
MRR: mean=0.21154299103948063, var=0.10798957880277361
Prec@(1) = 0.13 | Prec@(5) = 0.282 | Prec@(10) = 0.398 | Prec@(50) = 0.646 | Prec@(100) = 0.83 | Prec@(500) = 1.0 | NUMBER_OF_SA

### IDF ordered - softmaxed

In [ ]:
accs, preds = predict_labelwise_doc_at_history_ordered_softmaxed([test_queries[2],
                                                        test_queries[1],
                                                        test_queries[0]],
                                                        k=5)
print(accs)
print(preds)
print('-' * 20)

[0.37505364 0.37495323 0.36257429 0.36173653 0.36112785]
['Benefits Planner: Survivors | How You Apply | Social Security Administration#1_0', 'Benefits Planner: Survivors | How You Apply | Social Security Administration#2_0', 'Benefits Planner: Disability | How You Apply | Social Security Administration#2_0', 'Learn About Retirement Benefits | SSA#1_0', 'Benefits Planner: Disability | How You Apply | Social Security Administration#1_0']
--------------------


In [ ]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in range(2, TEST_SIZE):
    act_doc = doc_label_test[i]
    accs, preds = predict_labelwise_doc_at_history_ordered_softmaxed([doc_sentence_test[i],
                                                            doc_sentence_test[i-1],
                                                            doc_sentence_test[i-2]],
                                                            k=500)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

MRR: mean=0.09269418454220091, var=0.041466296907041615
Prec@(1) = 0.04 | Prec@(5) = 0.12 | Prec@(10) = 0.24 | Prec@(50) = 0.44 | Prec@(100) = 0.6 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100
MRR: mean=0.090842020703304, var=0.042118003791540175
Prec@(1) = 0.04 | Prec@(5) = 0.11 | Prec@(10) = 0.21 | Prec@(50) = 0.44 | Prec@(100) = 0.62 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200
MRR: mean=0.11415570292589938, var=0.05874265241044486
Prec@(1) = 0.06 | Prec@(5) = 0.14 | Prec@(10) = 0.24666666666666667 | Prec@(50) = 0.4633333333333333 | Prec@(100) = 0.6766666666666666 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 300
MRR: mean=0.14876841730908613, var=0.08039533161912413
Prec@(1) = 0.0875 | Prec@(5) = 0.19 | Prec@(10) = 0.29 | Prec@(50) = 0.5175 | Prec@(100) = 0.7275 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 400
MRR: mean=0.16919351545310174, var=0.09242517954525063
Prec@(1) = 0.104 | Prec@(5) = 0.22 | Prec@(10) = 0.306 | Prec@(50) = 0.546 | Prec@(100) = 0.742 | Prec@(500) = 1.0 | NUMBER_OF_SA

### IDF + self attention (cosine sim)

In [ ]:
accs, preds = predict_labelwise_doc_at_history_selfatt([test_queries[2],
                                                        test_queries[1],
                                                        test_queries[0]],
                                                        k=5)
print(accs)
print(preds)
print('-' * 20)

[0.4257137  0.42203476 0.42132395 0.41824857 0.4159181 ]
['Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#7_0_1_2', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3_4', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#12_0_1_2']
--------------------


In [ ]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in range(2, TEST_SIZE):
    act_doc = doc_label_test[i]
    accs, preds = predict_labelwise_doc_at_history_selfatt([doc_sentence_test[i],
                                                            doc_sentence_test[i-1],
                                                            doc_sentence_test[i-2]],
                                                            k=500)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

MRR: mean=0.13712179858232124, var=0.06804061197193524
Prec@(1) = 0.07 | Prec@(5) = 0.17 | Prec@(10) = 0.31 | Prec@(50) = 0.49 | Prec@(100) = 0.72 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100
MRR: mean=0.1407645002818882, var=0.07746681079191914
Prec@(1) = 0.085 | Prec@(5) = 0.165 | Prec@(10) = 0.28 | Prec@(50) = 0.505 | Prec@(100) = 0.755 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200
MRR: mean=0.16801966978901922, var=0.08910630754713314
Prec@(1) = 0.1 | Prec@(5) = 0.22333333333333333 | Prec@(10) = 0.33 | Prec@(50) = 0.5266666666666666 | Prec@(100) = 0.7733333333333333 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 300
MRR: mean=0.17874436936684396, var=0.09063010654135949
Prec@(1) = 0.1025 | Prec@(5) = 0.2475 | Prec@(10) = 0.355 | Prec@(50) = 0.5775 | Prec@(100) = 0.795 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 400
MRR: mean=0.20205575728498712, var=0.10276163124862168
Prec@(1) = 0.122 | Prec@(5) = 0.274 | Prec@(10) = 0.388 | Prec@(50) = 0.618 | Prec@(100) = 0.824 | Prec@(500) = 1.0 | NUMBER

### DR.TEIT

In [1]:

prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
l = len(test_df)
for i in range(2, l):
    act_doc = test_df.loc[i].current_doc
   # print(test_df.loc[i])
    preds = predict_FUDNet_DR_TEIT(test_df.loc[i], k=35659)
    pred = []
    for i in range(len(preds)):
       ind = titles.index(preds[i])
      # print(ind)
       pred.append(parent_titles[ind])
    #ranks.append(1 / (pred.index(act_doc) + 1))
    if act_doc == pred[0]:
        prec_at_1 += 1
    if act_doc in pred[:5]:
        prec_at_5 += 1
    if act_doc in pred[:10]:
        prec_at_10 += 1
    if act_doc in pred[:50]:
        prec_at_50 += 1
    if act_doc in pred[:100]:
        prec_at_100 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
       # print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now,
                     sample_till_now))

NameError: ignored

## Results

At last we have resutls as follows:


| Method | @1 | @5 | @10 | @50 | @100 | MRR (mean, var) |
|:------:|:------:|:------:|:-------:|:-------:|:--------:|:---:|
| IDF - vanilla | 13% | 30% | 39% | 64% | 83% | (0.22, 0.11) |
| IDF - power-order | 15% | 31% | 41% | 65% | 83% | (0.23, 0.12) |
| IDF - power-order (softmax) | 10.7% | 23% | 31% | 57.6% | 78% | (0.18, 0.09) |
| IDF - self-attention | 13.9% | 29% | 38% | 62% | 82% | (0.22, 0.11) |
| **DR. TEIT** | **61.6%** | **86%** | **91%** | **96%** | **98%** | **(0.72, 0.13)** |

It shows that title informations were not enough for document retrieval.

# drafts

In [ ]:
tfidf_wm.shape

(488, 1047632)

In [ ]:
answers = tfidfVectorizer.transform(["Original Card for a Foreign Born U.S. Citizen Adult",
                                     "Hello world from far beyound!"]).todense()
query = tfidfVectorizer.transform(["Hello!"]).todense()

In [ ]:
print(answers.shape, query.shape)

(2, 1047632) (1, 1047632)


In [ ]:
import numpy as np
answers_sim = np.squeeze(np.asarray(tfidf_wm @ answers.T))
query_sim = np.squeeze(np.asarray(tfidf_wm @ query.T))

In [ ]:
print(answers_sim.shape, query_sim.shape)

(488, 2) (488,)


In [ ]:
list(map(lambda x: np.dot(x, query_sim) /
        (np.linalg.norm(query_sim) * np.linalg.norm(x)),
        answers_sim.T))

[0.7506911990367025, 0.934114716518692]